# Sheet 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import pyplot as plt
%matplotlib inline

## 1 Principal Component Analysis
### (a)

In [18]:
data = np.array([[1, 2, 3],[4, 5, 6]])
data

array([[1, 2, 3],
       [4, 5, 6]])

In [51]:
X

array([[-1.,  0.,  1.],
       [-1.,  0.,  1.]])

In [50]:
X_cov

array([[2., 2.],
       [2., 2.]])

In [49]:
np.linalg.eig(X_cov)

(array([4.0000000e+00, 4.4408921e-16]),
 array([[ 0.70710678, -0.70710678],
        [ 0.70710678,  0.70710678]]))

In [59]:
eigenvec[:,e_indices]

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])

In [58]:
np.argsort(eigenval)[::-1]

array([0, 1])

In [61]:
p,n = data.shape #[row,column]
X = data - (1/n)*(data@np.ones((n,1))@np.ones((1,n)))
X_cov = X@X.T  
eigenval, eigenvec = np.linalg.eig(X_cov)
e_indices = np.argsort(eigenval)[::-1]
eigenvec_sorted = eigenvec[:,e_indices]
eigenvec_sorted.T

array([[ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])

In [ ]:
# TODO: implement PCA (fill in the blanks in the function below)

def pca(data, n_components=None):
    """
    Principal Component Analysis on a p x N data matrix.
    
    Parameters
    ----------
    data : np.ndarray
        Data matrix of shape (p, N).
    n_components : int, optional
        Number of requested components. By default returns all components.
        
    Returns
    -------
    np.ndarray, np.ndarray
        the pca components (shape (n_components, p)) and the projection (shape (n_components, N))

    """
    # set n_components to p by default
    n_components = data.shape[0] if n_components is None else n_components
    assert n_components <= data.shape[0], f"Got n_components larger than dimensionality of data!"
    
    # center the data
    p = n_components
    n = data.shape[1] #[row,column]
    X = data - (1/n)*(data@np.ones((n,1))@np.ones((1,n)))

    # compute X times X transpose
    X_cov = X@X.T                       # pxp matrix
    
    # compute the eigenvectors and eigenvalues
    eigenval, eigenvec = np.linalg.eig(X_cov)

    # sort the eigenvectors by eigenvalue and take the n_components largest ones
    e_indices = np.argsort(eigenval)[::-1][:p]
    eigenvec_sorted = eigenvec[:,e_indices]

    # compute X_projected, the projection of the data to the components
    components = eigenvec_sorted.T
    
    return components, X_projected  # return the n_components first components and the pca projection of the data


In [ ]:
# Example data to test your implementation 
# All the asserts on the bottom should go through if your implementation is correct

data = np.array([
    [ 1,  0,  0, -1,  0,  0],
    [ 0,  3,  0,  0, -3,  0],
    [ 0,  0,  5,  0,  0, -5]
], dtype=np.float32)

# add a random offset to all samples. it should not affect the results
data += np.random.randn(data.shape[0], 1)

n_components = 2
components, projection = pca(data, n_components=n_components)  # apply your implementation

# the correct results are known (up to some signs)
true_components = np.array([[0, 0, 1], [0, 1, 0]], dtype=np.float32)
true_projection = np.array([
    [ 0,  0,  5,  0,  0, -5],
    [ 0,  3,  0,  0, -3,  0]
], dtype=np.float32)

# check that components match, up to sign
assert isinstance(components, np.ndarray), f'Expected components to be numpy array but got {type(components)}'
assert components.shape == true_components.shape, f'{components.shape}!={true_components.shape}'
assert np.allclose(np.abs(components * true_components).sum(1), np.ones(n_components)), f'Components not matching'

# check that projections agree, taking into account potentially flipped components
assert isinstance(projection, np.ndarray), f'Expected projection to be numpy array but got {type(projection)}'
assert projection.shape == (n_components, data.shape[1]), f'Incorrect shape of projection: Expected {(n_components, data.shape[1])}, got {projection.shape}'
assert np.allclose(projection, true_projection * (components * true_components).sum(1, keepdims=True), atol=1e-6), f'Projections not matching'

print('Test successful!')

### (b)

Load the data (it is a subset of the data at https://opendata.cern.ch/record/4910#)

In [ ]:
features = np.load('data/dijet_features.npy')
labels = np.load('data/dijet_labels.npy')
label_names = ['b', 'c', 'q']  # bottom, charm or light quarks

print(f'{features.shape=}, {labels.shape=}')  # print the shapes

# TODO: print how many samples of each class are present in the data (hint: numpy.unique)


Normalize the data

In [ ]:
# TODO: subtract the mean of each feature to center the data

# TODO: divide by the standard deviation of each feature to normalize the variance


### (c)
Compute a 2D PCA projection and make a scatterplot of the result, once without color, once coloring the dots by label

In [ ]:
# TODO: apply PCA as implemented in (a)


In [ ]:
# TODO: make a scatterplot of the PCA projection


In [ ]:
# TODO: make a scatterplot, coloring the dots by their label and including a legend with the label names
# (hint: one way is to call plt.scatter once for each of the three possible labels)


## 2 Nonlinear Dimension Reduction

In [ ]:
import umap  # import umap-learn, see https://umap-learn.readthedocs.io/

In [ ]:
# if you have not done 1(b) yet, you can load the normalized features directly:
features = np.load('data/dijet_features_normalized.npy')
labels = np.load('data/dijet_labels.npy')
label_names = ['b', 'c', 'q']  # bottom, charm or light quarks

### (a)

In [ ]:
# TODO: Apply umap on the normalized jet features from excercise 1. It will take a couple of seconds.
# note: umap uses a different convention regarding the feature- and sample dimension, N x p instead of p x N!

reducer = umap.UMAP()

In [ ]:
# TODO: make a scatterplot of the UMAP projection

# TODO: make a scatterplot, coloring the dots by their label and including a legend with the label names
# (hint: one way is to call plt.scatter once for each of the three possible labels)


### (b)

In [ ]:
for n_neighbors in (2, 4, 8, 15, 30, 60, 100):
    # TODO: repeat the above, varying the n_neighbors parameter of UMAP
    reducer = umap.UMAP(n_neighbors=n_neighbors)
    

### (c)

In [ ]:
for n_components in (2, 4, 8, 16, 32, 64, len(features)):
    # TODO: project to the n-components first principal components 
    #       (use your implementation from ex. 1 or PCA from scikit-learn)
    
    # TODO: apply UMAP to get from n_components to just two dimensions
    
    # TODO: again, make scatterplots as before